In [2]:
pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\fbwod\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
pip install scikit-learn

     --------------------------------------- 11.1/11.1 MB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 41.3/41.3 MB 7.4 MB/s eta 0:00:00
     -------------------------------------- 307.7/307.7 KB 9.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\fbwod\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
import os
import shutil
import torch
import torch.nn as nn
import numpy as np
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import log_loss




In [6]:
# 하이퍼파라미터
num_classes = 396
batch_size = 16
epochs = 10
lr = 3e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 경로 설정
base_path = "C:/Users/fbwod/Desktop/DL_Oldcar"
train_path = os.path.join(base_path, "train")
val_path = os.path.join(base_path, "val")



In [7]:
# ✅ val 폴더 생성 (한 번만 실행됨)
if not os.path.exists(val_path):
    print("🔧 val/ 폴더 생성 중 (train에서 10% 분리)...")
    class_names = os.listdir(train_path)
    image_paths, labels = [], []
    for cls in class_names:
        cls_path = os.path.join(train_path, cls)
        files = [f for f in os.listdir(cls_path) if f.endswith(('.jpg', '.png'))]
        for f in files:
            image_paths.append(os.path.join(cls_path, f))
            labels.append(cls)

    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
    _, val_idx = next(splitter.split(image_paths, labels))

    for i in val_idx:
        src = image_paths[i]
        cls = labels[i]
        dst_dir = os.path.join(val_path, cls)
        os.makedirs(dst_dir, exist_ok=True)
        shutil.copy(src, os.path.join(dst_dir, os.path.basename(src)))
    print("✅ val 데이터 분할 완료.")

# 전처리 정의
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# 데이터셋 로딩
train_dataset = ImageFolder(train_path, transform=train_transform)
val_dataset = ImageFolder(val_path, transform=val_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



In [10]:
# 초기화
best_loss = float('inf')
patience = 5  # 몇 epoch까지 개선이 없을 때 멈출지
counter = 0
save_path = 'best_model.pth'

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

        if i % 10 == 0:
            print(f"Epoch {epoch+1} | Batch {i+1}/{len(train_loader)} | Loss: {loss.item():.4f}")

    train_acc = correct / total

    # 🔍 Validation Accuracy + Log Loss
    model.eval()
    val_correct = 0
    val_total = 0
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)
            all_probs.append(probs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            val_correct += (outputs.argmax(1) == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    all_probs = np.concatenate(all_probs)
    all_labels = np.concatenate(all_labels)
    val_logloss = log_loss(all_labels, all_probs, labels=np.arange(num_classes))

    print(f"[Epoch {epoch+1}] Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Val LogLoss: {val_logloss:.4f}, Loss: {running_loss:.4f}")

    # ✅ Early Stopping 로직
    if val_logloss < best_loss:
        best_loss = val_logloss
        counter = 0
        torch.save(model.state_dict(), save_path)
        print(f"✅ Model improved. Saved to {save_path}")
    else:
        counter += 1
        print(f"⚠️ No improvement. EarlyStopping counter: {counter}/{patience}")
        if counter >= patience:
            print("⛔ Early stopping triggered.")
            break


Epoch 1 | Batch 1/2072 | Loss: 6.0656
Epoch 1 | Batch 11/2072 | Loss: 6.3511
Epoch 1 | Batch 21/2072 | Loss: 6.1276
Epoch 1 | Batch 31/2072 | Loss: 5.7313
Epoch 1 | Batch 41/2072 | Loss: 6.5180
Epoch 1 | Batch 51/2072 | Loss: 5.9967
Epoch 1 | Batch 61/2072 | Loss: 6.4439
Epoch 1 | Batch 71/2072 | Loss: 6.0497
Epoch 1 | Batch 81/2072 | Loss: 6.2242
Epoch 1 | Batch 91/2072 | Loss: 5.8233
Epoch 1 | Batch 101/2072 | Loss: 6.8509
Epoch 1 | Batch 111/2072 | Loss: 6.0142
Epoch 1 | Batch 121/2072 | Loss: 5.3857
Epoch 1 | Batch 131/2072 | Loss: 5.6796
Epoch 1 | Batch 141/2072 | Loss: 5.6563
Epoch 1 | Batch 151/2072 | Loss: 5.6854
Epoch 1 | Batch 161/2072 | Loss: 5.4866
Epoch 1 | Batch 171/2072 | Loss: 5.5678
Epoch 1 | Batch 181/2072 | Loss: 5.1941
Epoch 1 | Batch 191/2072 | Loss: 5.4829
Epoch 1 | Batch 201/2072 | Loss: 4.8271
Epoch 1 | Batch 211/2072 | Loss: 4.7423
Epoch 1 | Batch 221/2072 | Loss: 5.0510
Epoch 1 | Batch 231/2072 | Loss: 5.7995
Epoch 1 | Batch 241/2072 | Loss: 5.2103
Epoch 1 | B

In [11]:
# 모델 저장
torch.save(model.state_dict(), "resnet18_finetuned.pth")
print("✅ 모델 저장 완료: resnet18_finetuned.pth")

✅ 모델 저장 완료: resnet18_finetuned.pth
